In [135]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge , ElasticNet, LogisticRegression
from sklearn.model_selection import KFold , StratifiedKFold
from sklearn.model_selection import cross_val_predict,cross_val_score
import os
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector,make_column_transformer

In [159]:
df = pd.read_csv('train_loan.csv', index_col=0)

In [161]:
df

Gender Married Dependents     Education Self_Employed  \
Loan_ID                                                           
LP001002    Male      No          0      Graduate            No   
LP001003    Male     Yes          1      Graduate            No   
LP001005    Male     Yes          0      Graduate           Yes   
LP001006    Male     Yes          0  Not Graduate            No   
LP001008    Male      No          0      Graduate            No   
...          ...     ...        ...           ...           ...   
LP002978  Female      No          0      Graduate            No   
LP002979    Male     Yes         3+      Graduate            No   
LP002983    Male     Yes          1      Graduate            No   
LP002984    Male     Yes          2      Graduate            No   
LP002990  Female      No          0      Graduate           Yes   

          ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
Loan_ID                                                                      
LP001002             5849                0.0         NaN             360.0   
LP001003             4583             1508.0       128.0             360.0   
LP001005             3000                0.0        66.0             360.0   
LP001006             2583             2358.0       120.0             360.0   
LP001008             6000                0.0       141.0             360.0   
...                   ...                ...         ...               ...   
LP002978             2900                0.0        71.0             360.0   
LP002979             4106                0.0        40.0             180.0   
LP002983             8072              240.0       253.0             360.0   
LP002984             7583                0.0       187.0             360.0   
LP002990             4583                0.0       133.0             360.0   

          Credit_History Property_Area Loan_Status  
Loan_ID                                             
LP001002             1.0         Urban           Y  
LP001003             1.0         Rural           N  
LP001005             1.0         Urban           Y  
LP001006             1.0         Urban           Y  
LP001008             1.0         Urban           Y  
...                  ...           ...         ...  
LP002978             1.0         Rural           Y  
LP002979             1.0         Rural           Y  
LP002983             1.0         Urban           Y  
LP002984             1.0         Urban           Y  
LP002990             0.0     Semiurban           N  

[614 rows x 12 columns]

In [163]:
df.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [165]:
X , y = df.drop('Loan_Status', axis = 1), df['Loan_Status']
X

Gender Married Dependents     Education Self_Employed  \
Loan_ID                                                           
LP001002    Male      No          0      Graduate            No   
LP001003    Male     Yes          1      Graduate            No   
LP001005    Male     Yes          0      Graduate           Yes   
LP001006    Male     Yes          0  Not Graduate            No   
LP001008    Male      No          0      Graduate            No   
...          ...     ...        ...           ...           ...   
LP002978  Female      No          0      Graduate            No   
LP002979    Male     Yes         3+      Graduate            No   
LP002983    Male     Yes          1      Graduate            No   
LP002984    Male     Yes          2      Graduate            No   
LP002990  Female      No          0      Graduate           Yes   

          ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
Loan_ID                                                                      
LP001002             5849                0.0         NaN             360.0   
LP001003             4583             1508.0       128.0             360.0   
LP001005             3000                0.0        66.0             360.0   
LP001006             2583             2358.0       120.0             360.0   
LP001008             6000                0.0       141.0             360.0   
...                   ...                ...         ...               ...   
LP002978             2900                0.0        71.0             360.0   
LP002979             4106                0.0        40.0             180.0   
LP002983             8072              240.0       253.0             360.0   
LP002984             7583                0.0       187.0             360.0   
LP002990             4583                0.0       133.0             360.0   

          Credit_History Property_Area  
Loan_ID                                 
LP001002             1.0         Urban  
LP001003             1.0         Rural  
LP001005             1.0         Urban  
LP001006             1.0         Urban  
LP001008             1.0         Urban  
...                  ...           ...  
LP002978             1.0         Rural  
LP002979             1.0         Rural  
LP002983             1.0         Urban  
LP002984             1.0         Urban  
LP002990             0.0     Semiurban  

[614 rows x 11 columns]

In [167]:
si = SimpleImputer(strategy='constant', fill_value='Unknown').set_output(transform='pandas')
X[['Gender','Married','Dependents','Self_Employed']] = si.fit_transform(X[['Gender','Married','Dependents','Self_Employed']])

In [169]:
si2= SimpleImputer(strategy='median').set_output(transform='pandas')
X[['LoanAmount','Loan_Amount_Term', 'Credit_History']] = si2.fit_transform(X[['LoanAmount','Loan_Amount_Term', 'Credit_History']])

In [171]:
X_ohe = OneHotEncoder(sparse_output=False,drop='first', handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer((X_ohe,make_column_selector(dtype_include=object)),
                            ('passthrough',make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X)
X_trn

Gender_Male  Gender_Unknown  Married_Unknown  Married_Yes  \
Loan_ID                                                               
LP001002          1.0             0.0              0.0          0.0   
LP001003          1.0             0.0              0.0          1.0   
LP001005          1.0             0.0              0.0          1.0   
LP001006          1.0             0.0              0.0          1.0   
LP001008          1.0             0.0              0.0          0.0   
...               ...             ...              ...          ...   
LP002978          0.0             0.0              0.0          0.0   
LP002979          1.0             0.0              0.0          1.0   
LP002983          1.0             0.0              0.0          1.0   
LP002984          1.0             0.0              0.0          1.0   
LP002990          0.0             0.0              0.0          0.0   

          Dependents_1  Dependents_2  Dependents_3+  Dependents_Unknown  \
Loan_ID                                                                   
LP001002           0.0           0.0            0.0                 0.0   
LP001003           1.0           0.0            0.0                 0.0   
LP001005           0.0           0.0            0.0                 0.0   
LP001006           0.0           0.0            0.0                 0.0   
LP001008           0.0           0.0            0.0                 0.0   
...                ...           ...            ...                 ...   
LP002978           0.0           0.0            0.0                 0.0   
LP002979           0.0           0.0            1.0                 0.0   
LP002983           1.0           0.0            0.0                 0.0   
LP002984           0.0           1.0            0.0                 0.0   
LP002990           0.0           0.0            0.0                 0.0   

          Education_Not Graduate  Self_Employed_Unknown  Self_Employed_Yes  \
Loan_ID                                                                      
LP001002                     0.0                    0.0                0.0   
LP001003                     0.0                    0.0                0.0   
LP001005                     0.0                    0.0                1.0   
LP001006                     1.0                    0.0                0.0   
LP001008                     0.0                    0.0                0.0   
...                          ...                    ...                ...   
LP002978                     0.0                    0.0                0.0   
LP002979                     0.0                    0.0                0.0   
LP002983                     0.0                    0.0                0.0   
LP002984                     0.0                    0.0                0.0   
LP002990                     0.0                    0.0                1.0   

          Property_Area_Semiurban  Property_Area_Urban  ApplicantIncome  \
Loan_ID                                                                   
LP001002                      0.0                  1.0             5849   
LP001003                      0.0                  0.0             4583   
LP001005                      0.0                  1.0             3000   
LP001006                      0.0                  1.0             2583   
LP001008                      0.0                  1.0             6000   
...                           ...                  ...              ...   
LP002978                      0.0                  0.0             2900   
LP002979                      0.0                  0.0             4106   
LP002983                      0.0                  1.0             8072   
LP002984                      0.0                  1.0             7583   
LP002990                      1.0                  0.0             4583   

          CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  
Loan_ID                                    

In [173]:
lr = LogisticRegression()
scl_std , scl_min = StandardScaler(), MinMaxScaler()
pipe = Pipeline([('scaler',None),('LR' , lr)])
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)
pipe.get_params()

{'memory': None,
 'steps': [('scaler', None), ('LR', LogisticRegression())],
 'transform_input': None,
 'verbose': False,
 'scaler': None,
 'LR': LogisticRegression(),
 'LR__C': 1.0,
 'LR__class_weight': None,
 'LR__dual': False,
 'LR__fit_intercept': True,
 'LR__intercept_scaling': 1,
 'LR__l1_ratio': None,
 'LR__max_iter': 100,
 'LR__multi_class': 'deprecated',
 'LR__n_jobs': None,
 'LR__penalty': 'l2',
 'LR__random_state': None,
 'LR__solver': 'lbfgs',
 'LR__tol': 0.0001,
 'LR__verbose': 0,
 'LR__warm_start': False}

In [175]:
params = {'LR__penalty': ['l1','l2','elasticnet',None] , 'scaler': [scl_std , scl_min, None],'LR__l1_ratio':[0.01, 0.5,1]}
gvc = GridSearchCV(pipe, param_grid=params,cv = kfold, scoring='accuracy')

In [177]:
gvc.fit(X_trn,y)

C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio paramet

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('scaler', None),
                                       ('LR', LogisticRegression())]),
             param_grid={'LR__l1_ratio': [0.01, 0.5, 1],
                         'LR__penalty': ['l1', 'l2', 'elasticnet', None],
                         'scaler': [StandardScaler(), MinMaxScaler(), None]},
             scoring='accuracy')

In [178]:
gvc.best_params_

{'LR__l1_ratio': 0.01, 'LR__penalty': None, 'scaler': None}

In [179]:
gvc.best_score_

0.8127282420365187

## Inferencing

In [206]:
test = pd.read_csv('test_loan.csv', index_col=0)
test.isnull().sum()

Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [192]:
si = SimpleImputer(strategy='constant', fill_value='Unknown').set_output(transform='pandas')
test[['Gender','Married','Dependents','Self_Employed']] = si.fit_transform(test[['Gender','Married','Dependents','Self_Employed']])

In [222]:
si2= SimpleImputer(strategy='median').set_output(transform='pandas')
test[['LoanAmount','Loan_Amount_Term', 'Credit_History']] = si2.fit_transform(test[['LoanAmount','Loan_Amount_Term', 'Credit_History']])

In [218]:
test

Gender Married Dependents     Education Self_Employed  \
Loan_ID                                                          
LP001015   Male     Yes          0      Graduate            No   
LP001022   Male     Yes          1      Graduate            No   
LP001031   Male     Yes          2      Graduate            No   
LP001035   Male     Yes          2      Graduate            No   
LP001051   Male      No          0  Not Graduate            No   
...         ...     ...        ...           ...           ...   
LP002971   Male     Yes         3+  Not Graduate           Yes   
LP002975   Male     Yes          0      Graduate            No   
LP002980   Male      No          0      Graduate            No   
LP002986   Male     Yes          0      Graduate            No   
LP002989   Male      No          0      Graduate           Yes   

          ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
Loan_ID                                                                      
LP001015             5720                  0       110.0             360.0   
LP001022             3076               1500       126.0             360.0   
LP001031             5000               1800       208.0             360.0   
LP001035             2340               2546       100.0             360.0   
LP001051             3276                  0        78.0             360.0   
...                   ...                ...         ...               ...   
LP002971             4009               1777       113.0             360.0   
LP002975             4158                709       115.0             360.0   
LP002980             3250               1993       126.0             360.0   
LP002986             5000               2393       158.0             360.0   
LP002989             9200                  0        98.0             180.0   

          Credit_History Property_Area  
Loan_ID                                 
LP001015             1.0         Urban  
LP001022             1.0         Urban  
LP001031             1.0         Urban  
LP001035             1.0         Urban  
LP001051             1.0         Urban  
...                  ...           ...  
LP002971             1.0         Urban  
LP002975             1.0         Urban  
LP002980             1.0     Semiurban  
LP002986             1.0         Rural  
LP002989             1.0         Rural  

[367 rows x 11 columns]

In [224]:
X_tst = ct.transform(test)

C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0, 2, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [253]:
y_pred = gvc.predict(X_tst)

In [255]:
submit = pd.read_csv('sample_submission.csv')

In [257]:
submit['Loan_Status'] = y_pred

In [259]:
submit

Loan_ID Loan_Status
0    LP001015           Y
1    LP001022           Y
2    LP001031           Y
3    LP001035           Y
4    LP001051           Y
..        ...         ...
362  LP002971           Y
363  LP002975           Y
364  LP002980           Y
365  LP002986           Y
366  LP002989           Y

[367 rows x 2 columns]

In [263]:
X = submit.to_csv('submit3.csv', index=False)